# Numerical Optimization and Large Scale Linear Algebra
## Project 3: PageRank Algorithm Implementation and Analysis

In [34]:
import numpy as np
from scipy.sparse import csr_matrix
import zipfile
import os
import time

from scipy.sparse.linalg import spsolve
from scipy.sparse import identity

## Data Loading and Preprocessing

We begin by loading the Stanford web connectivity data, which represents the hyperlink structure between Stanford University webpages. The data consists of directed edges where each row contains a source node and target node, representing a hyperlink from one page to another.

The preprocessing step creates a zero-indexed mapping of all unique nodes to ensure efficient matrix operations in subsequent calculations.

In [35]:
# Load web connectivity data from zip file and create node mappings
def load_web_data(filename):
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_file:
            dat_filename = zip_file.namelist()[0]
            with zip_file.open(dat_filename) as f:
                data = np.loadtxt(f, dtype=int)
    else:
        data = np.loadtxt(filename, dtype=int)
  
    sources = data[:, 0]
    targets = data[:, 1]
    
    all_nodes = np.unique(np.concatenate([sources, targets]))
    n = len(all_nodes)
    node_to_idx = {node: i for i, node in enumerate(all_nodes)}
    
    sources_idx = np.array([node_to_idx[s] for s in sources])
    targets_idx = np.array([node_to_idx[t] for t in targets])
    
    return sources_idx, targets_idx, n, all_nodes

sources, targets, n, node_list = load_web_data('stanweb.dat.zip')
print(f"{len(sources)} edges, {n} nodes")

/var/folders/27/w24km3fs2vl7bm7hlvrwm2l40000gn/T/ipykernel_61915/2698559121.py:7: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  data = np.loadtxt(f, dtype=int)


2382912 edges, 281903 nodes


## Transition Matrix Construction

The PageRank algorithm requires constructing a sparse transition matrix **P** from the web graph structure. Each entry P[i,j] represents the probability of transitioning from page i to page j.

- **Outlink normalization**: Each row sums to 1 by dividing by the number of outgoing links
- **Dangling nodes**: Pages with no outlinks are identified in vector **a** for special handling
- **Sparse representation**: Using CSR format for memory efficiency with large graphs

In [36]:
# Build sparse transition matrix P and dangling nodes vector a
def build_transition_matrix(sources, targets, n):
    outlinks = np.bincount(sources, minlength=n)
    a = (outlinks == 0).astype(float)
    weights = 1.0 / outlinks[sources]
    P = csr_matrix((weights, (sources, targets)), shape=(n, n))
    return P, a

P, a = build_transition_matrix(sources, targets, n)
print(f"Matrix shape: {P.shape}")
print(f"Dangling nodes: {np.sum(a):.0f}")

Matrix shape: (281903, 281903)
Dangling nodes: 172


## Part A: PageRank via Power Method

The power method is the traditional approach for computing PageRank. It iteratively applies the PageRank equation until convergence:

**π^(k+1) = α π^(k) P + (α π^(k) a + (1-α)) v^T**

Where:
- **α = 0.85**: Damping factor balancing link structure vs. random teleportation
- **Convergence criterion**: ||π^(k+1) - π^(k)||₁ < 10⁻⁸
- **Matrix-free implementation**: Avoids storing the dense Google matrix

In [37]:
# Compute PageRank using power method implementation
def pagerank_power_method(P, a, alpha=0.85, tol=1e-8, max_iter=1000):
    n = P.shape[0]
    x = np.ones(n) / n
    
    for iteration in range(max_iter):
        x_old = x.copy()
        x_new = alpha * (x @ P)
        x_new += alpha * np.dot(x, a) / n
        x_new += (1 - alpha) / n
        x = x_new
        
        if np.linalg.norm(x - x_old, 1) < tol:
            return x, iteration + 1
    
    return x, max_iter

start_time = time.time()
pi_power, power_iterations = pagerank_power_method(P, a, alpha=0.85)
power_time = time.time() - start_time
print(f"{power_iterations} iterations, {power_time:.4f} seconds")

91 iterations, 0.4580 seconds


## Alternative: Linear System Formulation

Instead of the iterative power method, PageRank can be computed by solving the linear system directly:

**(I - αP^T)π = (1-α)/n e**

This approach:
- **Direct solution**: No iteration required, single solve operation
- **Sparse solver**: Uses efficient sparse linear algebra routines
- **Guaranteed convergence**: No iteration count dependency

In [38]:
# Compute PageRank by solving linear system (I - αP^T)π = (1-α)/n * e
def pagerank_linear_system(P, a, alpha=0.85):
    n = P.shape[0]
    I = identity(n, format='csr')
    A = I - alpha * P.T
    b = (1 - alpha) / n * np.ones(n)
    pi = spsolve(A, b)
    return pi / np.sum(pi)

start_time = time.time()
pi_linear = pagerank_linear_system(P, a, alpha=0.85)
linear_time = time.time() - start_time
print(f"{linear_time:.4f} seconds")

15.0613 seconds


## Method Comparison and Results

Comparing the computational efficiency and accuracy of both approaches provides insights into their practical trade-offs. The L1 norm difference measures numerical accuracy between methods.

The top-ranked pages represent the most "important" nodes in Stanford's web graph according to the PageRank algorithm.

In [39]:
# Compare results from both methods and display top-ranked pages
difference = np.linalg.norm(pi_power - pi_linear, 1)

print(f"Power method: {power_iterations} iterations, {power_time:.4f}s")
print(f"Linear system: {linear_time:.4f}s")
print(f"L1 difference: {difference:.2e}")

top_10_indices = np.argsort(pi_power)[-10:][::-1]
print(f"\nTop 10 PageRank nodes:")
for i, idx in enumerate(top_10_indices):
    print(f"{i+1:2d}. Node {node_list[idx]}: {pi_power[idx]:.8f}")

Power method: 91 iterations, 0.4580s
Linear system: 15.0613s
L1 difference: 1.06e-08

Top 10 PageRank nodes:
 1. Node 89073: 0.01130284
 2. Node 226411: 0.00928766
 3. Node 241454: 0.00829723
 4. Node 262860: 0.00302312
 5. Node 134832: 0.00300127
 6. Node 234704: 0.00257226
 7. Node 136821: 0.00245370
 8. Node 68889: 0.00243078
 9. Node 105607: 0.00239743
10. Node 69358: 0.00236400


## Part B: Impact of α = 0.99

Increasing α closer to 1.0 emphasizes the actual link structure over random teleportation, but significantly affects convergence properties. The subdominant eigenvalue of the Google matrix approaches 1, slowing power method convergence dramatically.

In [40]:
# Test convergence behavior with α = 0.99
start_time = time.time()
pi_power_99, power_iterations_99 = pagerank_power_method(P, a, alpha=0.99, max_iter=2000)
power_time_99 = time.time() - start_time

start_time = time.time()
pi_linear_99 = pagerank_linear_system(P, a, alpha=0.99)
linear_time_99 = time.time() - start_time

print(f"Power method: {power_iterations_99} iterations, {power_time_99:.4f}s")
print(f"Linear system: {linear_time_99:.4f}s")
print(f"Slowdown factor: {power_iterations_99/power_iterations:.1f}x")

Power method: 1392 iterations, 5.4780s
Linear system: 6.9281s
Slowdown factor: 15.3x


## Ranking Stability Analysis

Different α values can produce significantly different PageRank orderings. Higher α values give more weight to the actual hyperlink structure, potentially promoting different pages to prominence.

The overlap in top-50 rankings quantifies how sensitive the algorithm is to the damping parameter choice.

In [41]:
# Compare rankings between α = 0.85 and α = 0.99
top_50_indices_85 = np.argsort(pi_power)[-50:][::-1]
top_50_indices_99 = np.argsort(pi_power_99)[-50:][::-1]
common_top_50 = len(set(top_50_indices_85) & set(top_50_indices_99))

print(f"Common nodes in top 50: {common_top_50}/50")

print(f"\nTop 10 (α = 0.99):")
for i, idx in enumerate(top_50_indices_99[:10]):
    rank_in_85 = np.where(top_50_indices_85 == idx)[0]
    rank_85 = rank_in_85[0] + 1 if len(rank_in_85) > 0 else "Not in top 50"
    print(f"{i+1:2d}. Node {node_list[idx]}: {pi_power_99[idx]:.8f} (α=0.85 rank: {rank_85})")

Common nodes in top 50: 25/50

Top 10 (α = 0.99):
 1. Node 89073: 0.00918678 (α=0.85 rank: 1)
 2. Node 281772: 0.00911239 (α=0.85 rank: Not in top 50)
 3. Node 174665: 0.00768853 (α=0.85 rank: Not in top 50)
 4. Node 226411: 0.00451447 (α=0.85 rank: 2)
 5. Node 179645: 0.00407272 (α=0.85 rank: 21)
 6. Node 271409: 0.00387233 (α=0.85 rank: Not in top 50)
 7. Node 262860: 0.00348533 (α=0.85 rank: 4)
 8. Node 136821: 0.00282083 (α=0.85 rank: 7)
 9. Node 68889: 0.00279021 (α=0.85 rank: 8)
10. Node 77988: 0.00267625 (α=0.85 rank: Not in top 50)


## Part C: Component-wise Convergence Analysis

The power method doesn't converge uniformly across all PageRank components. Different nodes may reach their final values at different rates, depending on their position in the web graph structure.

This analysis tracks convergence behavior of high-importance vs. low-importance nodes to understand the algorithm's dynamics.

In [42]:
# Analyze component-wise convergence behavior
def analyze_convergence(P, a, alpha=0.85, tol=1e-8):
    n = P.shape[0]
    x = np.ones(n) / n
    convergence_history = []
    
    for iteration in range(500):
        x_old = x.copy()
        x_new = alpha * (x @ P)
        x_new += alpha * np.dot(x, a) / n
        x_new += (1 - alpha) / n
        x = x_new
        
        if iteration % 10 == 0:
            convergence_history.append(x.copy())
        
        if np.linalg.norm(x - x_old, 1) < tol:
            break
    
    return x, convergence_history, iteration + 1

pi_85, history_85, iters_85 = analyze_convergence(P, a, alpha=0.85)
pi_99, history_99, iters_99 = analyze_convergence(P, a, alpha=0.99)

top_5_nodes = np.argsort(pi_85)[-5:][::-1]
bottom_5_nodes = np.argsort(pi_85)[:5]
print(f"Highest importance: {[node_list[i] for i in top_5_nodes]}")
print(f"Lowest importance: {[node_list[i] for i in bottom_5_nodes]}")

Highest importance: [89073, 226411, 241454, 262860, 134832]
Lowest importance: [1, 100721, 216231, 216228, 23318]


## Convergence Insights

The analysis reveals fundamental differences between iterative and direct solution approaches, with important implications for large-scale PageRank computation.

In [43]:
# Analyze convergence speed for different node types
def convergence_analysis(history, final_pi, top_nodes, bottom_nodes, alpha):
    iterations = len(history)
    top_errors = np.zeros(iterations)
    bottom_errors = np.zeros(iterations)
    
    for i, pi_iter in enumerate(history):
        top_errors[i] = np.mean([abs(pi_iter[node] - final_pi[node]) / final_pi[node] 
                                for node in top_nodes])
        bottom_errors[i] = np.mean([abs(pi_iter[node] - final_pi[node]) / final_pi[node] 
                                  for node in bottom_nodes])
    
    top_converge = np.where(top_errors < 0.01)[0]
    bottom_converge = np.where(bottom_errors < 0.01)[0]
    
    top_iter = top_converge[0] * 10 if len(top_converge) > 0 else "No convergence"
    bottom_iter = bottom_converge[0] * 10 if len(bottom_converge) > 0 else "No convergence"
    
    print(f"α = {alpha} - Iterations to 1% accuracy:")
    print(f"  High importance nodes: {top_iter}")
    print(f"  Low importance nodes: {bottom_iter}")

convergence_analysis(history_85, pi_85, top_5_nodes, bottom_5_nodes, 0.85)
convergence_analysis(history_99, pi_99, top_5_nodes, bottom_5_nodes, 0.99)

α = 0.85 - Iterations to 1% accuracy:
  High importance nodes: 20
  Low importance nodes: 0
α = 0.99 - Iterations to 1% accuracy:
  High importance nodes: 230
  Low importance nodes: 30


## Part D: Link Farm Analysis

### D.1: Adding an Isolated Page

When adding a new page X with no incoming or outgoing links to an existing web graph, the PageRank redistribution follows mathematical principles. The new page becomes a "dangling node" that only receives PageRank through the teleportation component.

In [44]:
# D.1: Mathematical analysis of adding isolated page X
n_orig = n
print(f"Adding isolated page X:")
print(f"Original pages: {n_orig/(n_orig+1):.6f} × original PageRank")
print(f"New page X: {1/(n_orig+1):.6f}")

Adding isolated page X:
Original pages: 0.999996 × original PageRank
New page X: 0.000004


### D.2: Strategic Link Addition

Creating page Y that links exclusively to X demonstrates how incoming links can boost PageRank. The improvement factor quantifies the benefit of having an incoming link versus remaining completely isolated.

In [45]:
# D.2: Adding page Y that links to X
def analyze_Y_links_to_X(n_original, alpha=0.85):
    y_pagerank = (1-alpha) / (n_original + 2)
    x_pagerank = (1-alpha) / (n_original + 2) + alpha * y_pagerank
    isolated_x_rank = 1 / (n_original + 2)
    
    print(f"Y PageRank: {y_pagerank:.8f}")
    print(f"X PageRank: {x_pagerank:.8f}")
    print(f"Improvement factor: {x_pagerank / isolated_x_rank:.3f}")

analyze_Y_links_to_X(n, alpha=0.85)

Y PageRank: 0.00000053
X PageRank: 0.00000098
Improvement factor: 0.278


### D.3-D.5: Link Farm Optimization Strategies

#### D.3: Optimal Link Structure for Three Pages (X, Y, Z)

The most effective configuration for maximizing page X's PageRank involves creating a **directed link farm**:

- **Y → X only** (Y has no other outlinks)
- **Z → X only** (Z has no other outlinks)  
- **X remains a dangling node** (no outlinks)

This structure concentrates all PageRank flow from Y and Z directly into X, maximizing the target page's importance score.

#### D.4: Counter-intuitive Effect of Outlinks

Adding outlinks from X to popular pages **reduces** X's PageRank due to the PageRank distribution mechanism:

- **PageRank dilution**: X's accumulated PageRank gets distributed among all its outlinks
- **Optimal strategy**: Keep X as a dangling node to retain all incoming PageRank
- **Trade-off**: While outlinks might improve user experience, they harm PageRank optimization

#### D.5: Scalable Link Farm Strategy

To maximize X's PageRank with unlimited resources:

1. **Create multiple farm pages** (Y, Z, W, ...) that link exclusively to X
2. **Maintain X as dangling node** to prevent PageRank leakage
3. **Scale effect**: Each additional farm page increases X's PageRank
4. **Mathematical contribution**: Each farm page adds approximately `(1-α)/n` teleportation PageRank
5. **Expected boost**: With `m` farm pages, X gains roughly `m × α × (1-α)/n` additional PageRank

This strategy exploits the PageRank algorithm's fundamental mechanics while remaining within the mathematical framework of the original model.